## 1. Importaciones y Configuración

Importamos las clases necesarias para trabajar con contenido multimodal:
- **ChatMessage**: Contenedor de mensajes
- **TextContent**: Para texto
- **DataContent**: Para datos binarios (imágenes)
- **Role**: Roles de los participantes (USER, ASSISTANT)
- **Path**: Para manejo de rutas de archivos

In [ ]:
import asyncio
from agent_framework import ChatMessage, TextContent, DataContent, Role
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential
from pathlib import Path
from dotenv import load_dotenv
import os

load_dotenv()

print("✅ Importaciones completadas")

## 2. Creación del Agente de Visión

### Configuración del Agente:

El agente de visión requiere:
- **Modelo compatible**: GPT-4 Vision (gpt-4o, gpt-4-vision, etc.)
- **Instructions**: Guía sobre cómo analizar imágenes
- **Credential**: Autenticación con Azure

### Importante:
- Asegúrate de que tu deployment en Azure OpenAI sea de un modelo con capacidades de visión
- Los modelos GPT-3.5 no soportan análisis de imágenes
- GPT-4o y GPT-4 Vision son las opciones recomendadas

In [ ]:
agent = AzureOpenAIChatClient(
    credential=AzureCliCredential(),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    deployment_name=os.getenv("MODEL"),  # Debe ser un modelo con capacidades de visión
).create_agent(
    name="VisionAgent",
    instructions="Eres un agente útil que puede analizar imágenes con gran detalle.",
)

print("✅ Agente de visión creado")
print(f"🤖 Modelo: {os.getenv('MODEL')}")
print("👁️  Capacidad: Análisis de imágenes y texto")

## 3. Carga de Imagen desde Archivo Local

### Proceso de Carga:

1. **Ubicar la imagen**: Usar Path para construir la ruta de forma portable
2. **Leer como binario**: Abrir en modo 'rb' (read binary)
3. **Almacenar bytes**: Los datos binarios se pasan al agente

### Formatos Soportados:
- JPEG (.jpg, .jpeg)
- PNG (.png)
- GIF (.gif)
- WebP (.webp)

### Consideraciones:
- Tamaño máximo: Típicamente 20MB
- Resolución: Mayor calidad = mejor análisis (pero más tokens)
- Costo: Imágenes de alta resolución consumen más tokens

In [ ]:
# Construir la ruta al archivo de imagen
script_dir = Path(".")  # Directorio actual (notebook)
img_path = script_dir / "images" / "Goku.jpg"

print(f"📂 Buscando imagen en: {img_path}")

# Verificar si el archivo existe
if img_path.exists():
    print(f"✅ Imagen encontrada")
    
    # Leer la imagen como bytes
    with open(img_path, "rb") as f:
        image_bytes = f.read()
    
    # Mostrar información sobre la imagen
    print(f"📊 Tamaño del archivo: {len(image_bytes) / 1024:.2f} KB")
else:
    print("❌ Imagen no encontrada. Asegúrate de que el archivo existe.")
    print(f"   Ruta esperada: {img_path.absolute()}")

## 4. Construcción del Mensaje Multimodal

### Estructura del Mensaje:

Un `ChatMessage` multimodal contiene:
- **role**: Quién envía el mensaje (USER, ASSISTANT, SYSTEM)
- **contents**: Lista de contenidos de diferentes tipos

### Tipos de Contenido:

1. **TextContent**: 
   - Contiene la pregunta o instrucción
   - Se especifica con `text=...`

2. **DataContent**: 
   - Contiene los datos binarios de la imagen
   - Requiere `data=...` (bytes)
   - Requiere `media_type=...` (ej: "image/jpeg", "image/png")

### Media Types Comunes:
- `image/jpeg` - Para archivos .jpg o .jpeg
- `image/png` - Para archivos .png
- `image/gif` - Para archivos .gif
- `image/webp` - Para archivos .webp

### Orden de Contenidos:
El orden puede ser importante:
- Texto primero + Imagen: "Analiza esta imagen..."
- Imagen primero + Texto: "[Imagen] ¿Qué ves aquí?"

Ambos funcionan, pero el primero suele ser más natural.

In [ ]:
# Construir mensaje con texto e imagen
message = ChatMessage(
    role=Role.USER,
    contents=[
        TextContent(text="¿Qué ves en esta imagen? Describe detalladamente."),
        DataContent(
            data=image_bytes,
            media_type="image/jpeg"  # Especificar el tipo MIME correcto
        )
    ]
)

print("✅ Mensaje multimodal construido")
print(f"📝 Componentes del mensaje:")
print(f"   - Texto: '{message.contents[0].text}'")
print(f"   - Imagen: {len(image_bytes)} bytes, tipo: {message.contents[1].media_type}")

## 5. Envío del Mensaje y Obtención de Respuesta

### Proceso de Análisis:

1. El agente recibe el mensaje multimodal
2. El modelo procesa la imagen y el texto
3. Genera una descripción basada en la comprensión visual
4. Retorna la respuesta en formato de texto

### Tiempos de Respuesta:
- Imágenes pequeñas: 2-5 segundos
- Imágenes grandes: 5-15 segundos
- Depende de la complejidad de la consulta

### Calidad del Análisis:
- El modelo puede identificar objetos, personas, escenas
- Reconoce texto visible en la imagen (OCR básico)
- Detecta colores, emociones, composición
- Puede hacer inferencias sobre contexto

In [ ]:
# Enviar mensaje al agente y obtener respuesta
print("🚀 Enviando imagen al agente para análisis...\n")

response = asyncio.run(agent.run(message))

print("💬 Respuesta del agente:")
print("=" * 80)
print(response.text)
print("=" * 80)

## 6. Consultas Adicionales sobre la Misma Imagen

Podemos hacer múltiples preguntas sobre la misma imagen:

In [ ]:
# Pregunta específica sobre colores
async def ask_about_colors():
    message = ChatMessage(
        role=Role.USER,
        contents=[
            TextContent(text="¿Qué colores predominan en la imagen?"),
            DataContent(data=image_bytes, media_type="image/jpeg")
        ]
    )
    response = await agent.run(message)
    print("🎨 Análisis de colores:")
    print(response.text)
    print()

await ask_about_colors()

In [ ]:
# Pregunta sobre el estado emocional
async def ask_about_emotion():
    message = ChatMessage(
        role=Role.USER,
        contents=[
            TextContent(text="¿Qué emociones o energía transmite el personaje?"),
            DataContent(data=image_bytes, media_type="image/jpeg")
        ]
    )
    response = await agent.run(message)
    print("😊 Análisis emocional:")
    print(response.text)
    print()

await ask_about_emotion()

In [ ]:
# Identificación de contexto
async def ask_about_context():
    message = ChatMessage(
        role=Role.USER,
        contents=[
            TextContent(text="¿Reconoces a este personaje? ¿De qué anime o manga es?"),
            DataContent(data=image_bytes, media_type="image/jpeg")
        ]
    )
    response = await agent.run(message)
    print("🎭 Identificación de personaje:")
    print(response.text)
    print()

await ask_about_context()

## 7. Análisis de Múltiples Imágenes

También podemos analizar múltiples imágenes en un solo mensaje para comparación:

In [ ]:
# Función auxiliar para analizar múltiples imágenes
async def compare_images(img_paths, question):
    """
    Analiza múltiples imágenes y responde una pregunta sobre ellas.
    
    Args:
        img_paths: Lista de rutas a las imágenes
        question: Pregunta sobre las imágenes
    """
    contents = [TextContent(text=question)]
    
    # Cargar cada imagen y agregarla al mensaje
    for img_path in img_paths:
        if Path(img_path).exists():
            with open(img_path, "rb") as f:
                img_bytes = f.read()
            
            # Detectar tipo MIME basado en extensión
            ext = Path(img_path).suffix.lower()
            media_type = {
                '.jpg': 'image/jpeg',
                '.jpeg': 'image/jpeg',
                '.png': 'image/png',
                '.gif': 'image/gif',
                '.webp': 'image/webp'
            }.get(ext, 'image/jpeg')
            
            contents.append(DataContent(data=img_bytes, media_type=media_type))
    
    message = ChatMessage(role=Role.USER, contents=contents)
    response = await agent.run(message)
    return response.text

print("✅ Función de comparación de imágenes definida")
print("💡 Uso: await compare_images(['img1.jpg', 'img2.jpg'], '¿En qué se parecen?')")

## 8. Casos de Uso Avanzados

### OCR (Reconocimiento de Texto)
Extraer texto visible en imágenes:

In [ ]:
async def extract_text_from_image(img_path):
    """
    Extrae texto de una imagen usando el agente de visión.
    Útil para documentos, capturas de pantalla, letreros, etc.
    """
    with open(img_path, "rb") as f:
        img_bytes = f.read()
    
    message = ChatMessage(
        role=Role.USER,
        contents=[
            TextContent(text="Extrae todo el texto visible en esta imagen. Mantén el formato original."),
            DataContent(data=img_bytes, media_type="image/jpeg")
        ]
    )
    
    response = await agent.run(message)
    return response.text

print("✅ Función de OCR definida")

### Análisis de Documentos
Interpretar gráficos, diagramas o tablas:

In [ ]:
async def analyze_chart(img_path):
    """
    Analiza gráficos, diagramas o visualizaciones de datos.
    El agente puede interpretar tendencias, valores y patrones.
    """
    with open(img_path, "rb") as f:
        img_bytes = f.read()
    
    message = ChatMessage(
        role=Role.USER,
        contents=[
            TextContent(text="Analiza este gráfico. Describe qué datos muestra, tendencias principales y conclusiones."),
            DataContent(data=img_bytes, media_type="image/png")
        ]
    )
    
    response = await agent.run(message)
    return response.text

print("✅ Función de análisis de gráficos definida")

## 9. Mejores Prácticas y Optimización

### Tips para Mejores Resultados:

1. **Calidad de Imagen**:
   - Usa imágenes claras y bien iluminadas
   - Evita imágenes borrosas o de muy baja resolución
   - Formato JPEG o PNG son ideales

2. **Tamaño de Imagen**:
   - Recomendado: 512x512 a 2048x2048 píxeles
   - Imágenes más grandes = más tokens = mayor costo
   - El modelo redimensiona automáticamente si es necesario

3. **Preguntas Específicas**:
   - Sé específico en lo que quieres saber
   - "Describe los colores" vs "¿Qué ves?"
   - Preguntas claras = respuestas más precisas

4. **Manejo de Errores**:
   ```python
   try:
       response = await agent.run(message)
   except Exception as e:
       print(f"Error al analizar imagen: {e}")
   ```

5. **Caché de Resultados**:
   - Guarda respuestas para evitar re-análisis
   - Usa hash de imagen como clave
   - Ahorra costos y tiempo

### Limitaciones a Considerar:

1. **Contenido Sensible**:
   - El modelo puede rechazar analizar contenido inapropiado
   - Tiene filtros de seguridad integrados

2. **Precisión**:
   - No es perfecto para OCR de documentos complejos
   - Puede tener alucinaciones (inventar detalles)
   - Verifica información crítica

3. **Idioma**:
   - Funciona mejor en inglés
   - Español funciona bien pero puede variar
   - Puedes especificar idioma de respuesta en instrucciones

4. **Costos**:
   - Imágenes consumen significativamente más tokens que texto
   - Alta resolución = más tokens
   - Monitorea uso para controlar costos

## 10. Análisis y Conclusiones

### Capacidades Demostradas:

1. **Análisis Visual Completo**:
   - Identificación de objetos y escenas
   - Descripción detallada de imágenes
   - Reconocimiento de contexto

2. **Multimodalidad**:
   - Combinar texto e imágenes en mensajes
   - Responder preguntas sobre contenido visual
   - Análisis comparativo de múltiples imágenes

3. **Flexibilidad**:
   - Diferentes tipos de consultas
   - Múltiples formatos de imagen
   - Análisis en varios idiomas

### Casos de Uso Prácticos:

1. **E-commerce**:
   - Descripción automática de productos
   - Categorización de imágenes
   - Búsqueda visual

2. **Moderación de Contenido**:
   - Detección de contenido inapropiado
   - Clasificación automática
   - Análisis de cumplimiento

3. **Accesibilidad**:
   - Generación de alt-text para imágenes
   - Descripción de gráficos para usuarios con discapacidad visual
   - Transcripción de texto en imágenes

4. **Educación**:
   - Análisis de diagramas y esquemas
   - Explicación de conceptos visuales
   - Ayuda con tareas que incluyen imágenes

5. **Medicina**:
   - Análisis preliminar de imágenes médicas (con disclaimers)
   - Documentación de casos
   - Asistencia en diagnóstico (supervisado)

6. **Real Estate**:
   - Descripción automática de propiedades
   - Análisis de características
   - Generación de listados

7. **Control de Calidad**:
   - Inspección visual automatizada
   - Detección de defectos
   - Verificación de conformidad

### Arquitectura para Producción:

```python
class ProductionVisionService:
    def __init__(self):
        self.agent = create_vision_agent()
        self.cache = ImageAnalysisCache()
        self.rate_limiter = RateLimiter(max_per_minute=60)
    
    async def analyze_image(self, img_bytes, question):
        # Check cache
        cache_key = hash_image(img_bytes)
        if cached := self.cache.get(cache_key, question):
            return cached
        
        # Rate limiting
        await self.rate_limiter.acquire()
        
        # Analyze
        result = await self.agent.run(build_message(img_bytes, question))
        
        # Cache result
        self.cache.set(cache_key, question, result)
        
        return result
```

### Integración con Otros Sistemas:

1. **Pipelines de Datos**:
   - Procesamiento batch de imágenes
   - ETL con enriquecimiento visual
   - Generación de metadatos

2. **APIs REST**:
   ```python
   @app.post("/analyze-image")
   async def analyze_endpoint(file: UploadFile, question: str):
       img_bytes = await file.read()
       result = await vision_service.analyze_image(img_bytes, question)
       return {"analysis": result}
   ```

3. **Eventos y Queues**:
   - Procesamiento asíncrono con Azure Service Bus
   - Workflows con Azure Logic Apps
   - Event-driven architecture

### Futuro de Vision AI:

- **Mayor precisión**: Modelos cada vez mejores
- **Menor latencia**: Procesamiento más rápido
- **Menor costo**: Optimización de recursos
- **Más modalidades**: Video, audio, 3D
- **Mejor razonamiento**: Comprensión contextual profunda